In [1]:
import findspark

In [2]:
findspark.init()

In [61]:
from pyspark.sql.functions import col
from pyspark.sql import functions as F

In [3]:
import pyspark
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder.appName('yxing_CBP').master('yarn-client').config(
        'spark.executor.memory', '28g').config('spark.cores.max', '10').config(
            'spark.driver.memory',
            '10g').config('spark.shuffle.consolidateFiles').config(
                'spark.executor.instances',
                '20').config('spark.yarn.queue',
                             'root.digital.heavy').getOrCreate())
sc = spark.sparkContext

In [68]:
prod_path = "/mapr/ia1.comscore.com/output/stream_sense_fact/239m/finalclose/c1_23/20191115/duration_deux/utc/streaming/valid"
test_path = "/mapr/ia1.comscore.com/user/tpires/dd2_twitch_20191115/valid"
schema = [
    "c1", "playlist_id", "clip_number", "content_identifier", "uid",
    "pattern_id", "time_id", "osid", "iab_flag", "ip_address", "ss2k_first",
    "url", "duration", "ns_ak_d_vce", "uid_hash", "raw_duration", "c2",
    "country", "is_nht", "waterfall_stats", "video_id", "ua_hash", "time_zone",
    "browser_type", "boc", "zip", "recorded", "publisher", "program",
    "episode", "content_type", "c3", "c4", "c6", "source_title_no",
    "source_owner_no", "source_genre_no"
]

In [87]:
len(schema)

37

In [69]:
prod = spark.read.format("csv").option("delimiter", '\t').option(
    "inferSchema", "true").load(prod_path).toDF(*schema)

In [10]:
test = spark.read.format("csv").option("delimiter", '\t').option(
    "inferSchema", "true").load(test_path).toDF(*schema)

In [19]:
# vmx_desk = prod.where(col('url').like("ns_ap_pn=web"))

In [70]:
twitch_tagless_desktop_prod = prod.filter((prod.url.contains('ns_ap_pn=web')) & (prod.c2=='31864766'))

In [71]:
twitch_tagless_desktop_prod.show()

+---+-----------+-----------+------------------+---+----------+-------+----+--------+----------+----------+---+--------+-----------+--------+------------+---+-------+------+---------------+--------+-------+---------+------------+---+---+--------+---------+-------+-------+------------+---+---+---+---------------+---------------+---------------+
| c1|playlist_id|clip_number|content_identifier|uid|pattern_id|time_id|osid|iab_flag|ip_address|ss2k_first|url|duration|ns_ak_d_vce|uid_hash|raw_duration| c2|country|is_nht|waterfall_stats|video_id|ua_hash|time_zone|browser_type|boc|zip|recorded|publisher|program|episode|content_type| c3| c4| c6|source_title_no|source_owner_no|source_genre_no|
+---+-----------+-----------+------------------+---+----------+-------+----+--------+----------+----------+---+--------+-----------+--------+------------+---+-------+------+---------------+--------+-------+---------+------------+---+---+--------+---------+-------+-------+------------+---+---+---+-----------

In [25]:
test.select("c2").describe().show()

+-------+-----------+
|summary|         c2|
+-------+-----------+
|  count|   76525463|
|   mean|3.1864766E7|
| stddev|        0.0|
|    min|   31864766|
|    max|   31864766|
+-------+-----------+



In [26]:
twitch_prod.printSchema()

root
 |-- c1: integer (nullable = true)
 |-- playlist_id: string (nullable = true)
 |-- clip_number: integer (nullable = true)
 |-- content_identifier: string (nullable = true)
 |-- uid: string (nullable = true)
 |-- pattern_id: integer (nullable = true)
 |-- time_id: integer (nullable = true)
 |-- osid: integer (nullable = true)
 |-- iab_flag: integer (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- ss2k_first: integer (nullable = true)
 |-- url: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- ns_ak_d_vce: integer (nullable = true)
 |-- uid_hash: long (nullable = true)
 |-- raw_duration: double (nullable = true)
 |-- c2: string (nullable = true)
 |-- country: string (nullable = true)
 |-- is_nht: integer (nullable = true)
 |-- waterfall_stats: long (nullable = true)
 |-- video_id: long (nullable = true)
 |-- ua_hash: string (nullable = true)
 |-- time_zone: double (nullable = true)
 |-- browser_type: integer (nullable = true)
 |-- boc: integer (nu

In [30]:
prod.printSchema()

root
 |-- c1: integer (nullable = true)
 |-- playlist_id: string (nullable = true)
 |-- clip_number: integer (nullable = true)
 |-- content_identifier: string (nullable = true)
 |-- uid: string (nullable = true)
 |-- pattern_id: integer (nullable = true)
 |-- time_id: integer (nullable = true)
 |-- osid: integer (nullable = true)
 |-- iab_flag: integer (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- ss2k_first: integer (nullable = true)
 |-- url: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- ns_ak_d_vce: integer (nullable = true)
 |-- uid_hash: long (nullable = true)
 |-- raw_duration: double (nullable = true)
 |-- c2: string (nullable = true)
 |-- country: string (nullable = true)
 |-- is_nht: integer (nullable = true)
 |-- waterfall_stats: long (nullable = true)
 |-- video_id: long (nullable = true)
 |-- ua_hash: string (nullable = true)
 |-- time_zone: double (nullable = true)
 |-- browser_type: integer (nullable = true)
 |-- boc: integer (nu

In [ ]:
# 1.	Valid streams are 55.1% of the total streams.
# 2.	I don’t have any device ID. I try to get this from URL parameter “ns_ak”, but these are always empty. I do have values for ns_ak_d_vce and ns_ak_d_aud. Should I fall back to these?
# 3.	isNHT flag is always -1
# 4.	videoId is always 0
# 5.	Birth of Cookie is always -1
# 6.	source_title_no, source_owner_no and source_genre_no are always 0



In [44]:
prod.select('url').show(20, False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [63]:
def get_details(df):
    r = {}
    r['country_content_type_dur_sum'] = df.groupBy(['country', 'content_type']).agg(F.sum('duration'))
    r['country_content_type_count'] = df.groupBy(['country', 'content_type']).count()
    r['country_countent_type_pattern_number'] = df.groupBy(['country', 'content_type']).agg(F.countDistinct('pattern_id'))
    r['is_nht_detail'] = df.select('is_nht').describe()
    r['video_id_detail'] = df.select('video_id').describe()
    r['birth_of_cookie_detail'] = df.select('boc').describe()
    r['nsak_detail'] = df.select('ns_ak_d_vce').describe()
    r['source_detail'] = df.select(['source_title_no','source_owner_no', 'source_genre_no']).describe()
    return r

In [64]:
test_result = get_details(test)

In [66]:
test_result['country_content_type_dur_sum'].show()

+-------+------------+-------------+
|country|content_type|sum(duration)|
+-------+------------+-------------+
|     GU|        vc22|    839736000|
|     WS|        vc23|     53868000|
|     LC|        va21|      3512132|
|     BM|        va21|     12031441|
|     BY|        vc23| 308943083966|
|     VC|        vc22|     36074000|
|     CM|        vc22|     14245000|
|     PR|        va11|      3140003|
|     GG|        vc22|    378478002|
|     GI|        vc23|   1967840001|
|     SV|        va21|     78265060|
|     MT|        vc23|  11192191002|
|     PF|        vc23|   1992552002|
|     UG|        vc22|     23271003|
|     AE|        va11|        22000|
|     NO|        vc22|  27096053040|
|     MY|        va11|     12324114|
|     A1|        vc23|   1755404000|
|     SR|        vc22|    142153000|
|     CO|        vc23| 202973165712|
+-------+------------+-------------+
only showing top 20 rows



In [67]:
test_result['country_content_type_count'].show()

+-------+------------+------+
|country|content_type| count|
+-------+------------+------+
|     GU|        vc22|   514|
|     WS|        vc23|    32|
|     LC|        va21|    63|
|     BM|        va21|   211|
|     BY|        vc23|262777|
|     CM|        vc22|    15|
|     VC|        vc22|    47|
|     GG|        vc22|   149|
|     PR|        va11|    78|
|     SV|        va21|  1863|
|     MT|        vc23|  8877|
|     PF|        vc23|  1557|
|     GI|        vc23|  1357|
|     UG|        vc22|    25|
|     AE|        va11|     1|
|     MY|        va11|   259|
|     NO|        vc22| 13649|
|     A1|        vc23|   831|
|     SR|        vc22|   119|
|     CO|        vc23|199939|
+-------+------------+------+
only showing top 20 rows



In [72]:
test_result['country_countent_type_pattern_number'].show()

+-------+------------+--------------------------+
|country|content_type|count(DISTINCT pattern_id)|
+-------+------------+--------------------------+
|     WS|        vc23|                         1|
|     LC|        va21|                         1|
|     GU|        vc22|                         1|
|     BM|        va21|                         1|
|     VC|        vc22|                         1|
|     BY|        vc23|                         1|
|     CM|        vc22|                         1|
|     PR|        va11|                         1|
|     GG|        vc22|                         1|
|     GI|        vc23|                         1|
|     MT|        vc23|                         1|
|     UG|        vc22|                         1|
|     SV|        va21|                         1|
|     AE|        va11|                         1|
|     PF|        vc23|                         1|
|     MY|        va11|                         1|
|     SR|        vc22|                         1|


In [73]:
test_result['is_nht_detail'].show()

+-------+--------+
|summary|  is_nht|
+-------+--------+
|  count|76525463|
|   mean|     0.0|
| stddev|     0.0|
|    min|       0|
|    max|       0|
+-------+--------+



In [82]:
test_result['video_id_detail'].show(10, False)

+-------+---------------------+
|summary|video_id             |
+-------+---------------------+
|count  |76525463             |
|mean   |2.819288293854335E14 |
|stddev |5.3246452407323689E18|
|min    |-9223371982240939063 |
|max    |9223371908799850607  |
+-------+---------------------+



In [75]:
test_result['birth_of_cookie_detail'].show()

+-------+--------+
|summary|     boc|
+-------+--------+
|  count|76525463|
|   mean|     0.0|
| stddev|     0.0|
|    min|       0|
|    max|       0|
+-------+--------+



In [76]:
test_result['source_detail'].show()

+-------+---------------+---------------+---------------+
|summary|source_title_no|source_owner_no|source_genre_no|
+-------+---------------+---------------+---------------+
|  count|       76525463|       76525463|       76525463|
|   mean|            0.0|            0.0|            0.0|
| stddev|            0.0|            0.0|            0.0|
|    min|              0|              0|              0|
|    max|              0|              0|              0|
+-------+---------------+---------------+---------------+



In [81]:
test_result['nsak_detail'].show(10,False)

+-------+----------------------------------------+
|summary|ns_ak_d_vce                             |
+-------+----------------------------------------+
|count  |76525463                                |
|mean   |null                                    |
|stddev |null                                    |
|min    |000006f8a725066651b14dd6b0bda8df2b0784e4|
|max    |null                                    |
+-------+----------------------------------------+



In [89]:
newColumns = [
    'c1', 'playlist_id', 'clip_number', 'content_identifier', 'uid',
    'pattern_id (dictionary not applied)', 'time_id', 'osid', 'iab',
    'ip_address', 'ss2k_first', 'url', 'duration', 'ns_ak_d_vce', 'uid_hash',
    'raw_duration', 'c2', 'country', 'is_nht', 'waterfall'
]

twitch_test_beacon_session = spark.read.format("csv").option(
    "delimiter", "\t"
).option("inferSchema", "true").option("header", "false").load(
    "/mapr/ia1.comscore.com/data/partitions/part_00000/beacon/monthly/239m/beacon_session/7271/c1=23/sorted_beacon_session_00000-10000"
)

twitch_test_beacon_session.show(5)

+---+----+--------------------+---+---------+--------+---+--------------+----------+---------+----+----+----+----+----+----+----+
|_c0| _c1|                 _c2|_c3|      _c4|     _c5|_c6|           _c7|       _c8|      _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|
+---+----+--------------------+---+---------+--------+---+--------------+----------+---------+----+----+----+----+----+----+----+
| 23|7271|$$cR6002238i66hhm...|  0|-26208573|70026707|  0|    46.18.62.2|1142692273|628265663| 0.0|   1|   0|  -1|  -1|  DE| 1.0|
| 23|7271|$$cR6002238i66hhm...|  0|-25980830|70026707|  0|    46.18.62.2|1142692273|628265670| 0.0|   1|   0|  -1|  -1|  DE| 1.0|
| 23|7271|$0H77m0k0D1n47hh5...|  0|-26208573|70134072|  0|93.218.169.170| 567235568|628282622| 0.0|   1|   0|  -1|  -1|  DE| 1.0|
| 23|7271|$14s8802226185hhB...|  0|-25980830|70117889|  0| 62.251.54.105|3032286467|628273594| 0.0|   1|   0|  -1|  -1|  NL| 1.0|
| 23|7271|$14s8802226185hhB...|  0|-25980830|70117889|  0| 62.251.54.105|3032286467|628291

In [ ]:
twitch_test_beacon_session = spark.read.format("csv").option(
    "delimiter", "\t"
).option("inferSchema", "true").option("header", "false").load(
    "/mapr/ia1.comscore.com/data/partitions/part_00000/beacon/monthly/239m/beacon_session/7271/c1=23/sorted_beacon_session_00000-10000"
)

# Assign Column Hearders
oldColumns = twitch_test_beacon_session.schema.names

newColumns = [
    'c1', 'playlist_id', 'clip_number', 'content_identifier', 'uid',
    'pattern_id (dictionary not applied)', 'time_id', 'osid', 'iab',
    'ip_address', 'ss2k_first', 'url', 'duration', 'ns_ak_d_vce', 'uid_hash',
    'raw_duration', 'c2', 'country', 'is_nht', 'waterfall'
]

twitch_test_beacon_session = reduce(
    lambda twitch_test_beacon_session, idx: twitch_test_beacon_session.
    withColumnRenamed(oldColumns[idx], newColumns[idx]),
    range(len(oldColumns)), twitch_test_beacon_session)
twitch_test_beacon_session.printSchema()